# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [4]:
# import libraries
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import numpy as np


In [5]:
# load messages dataset
messages = pd.read_csv('messages.csv', dtype = str)
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [81]:
messages['message'][0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [6]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [7]:
messages.head(2)

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [9]:
# merge 2datasets
df = pd.merge(messages, categories, left_index=True, right_index=True, how='outer')
df.head(2)

,id_x,message,original,genre,id_y,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,7,related-1;request-0;offer-0;aid_related-1;medi...


In [10]:
df.drop('id_y', axis=1, inplace=True )

In [11]:
df.shape

(26248, 5)

In [12]:
df.rename(columns={'id_x': 'id'}, inplace=True)

In [13]:
df.head(2)

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [14]:
categories.head(2)

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...


In [167]:
# categories2 = categories.copy(deep=True)

In [168]:
# for i in catnumcol.columns: 
#     x = catcol1[i].split('-')[0]
#     y = catcol1[i].split('-')[1]
#     categories2.loc[:,x] = ''

In [19]:
categories2.shape

NameError: name 'categories2' is not defined

In [20]:
# create a dataframe of the 36 individual category columns
categories = categories['categories'].str.split(";", expand=True)
categories.head()

KeyError: 'categories'

In [267]:
categories.shape

(26248, 36)

In [177]:
categories2.columns[5]

'aid_related'

In [21]:
row = categories.iloc[0]
row

0                    related-1
1                    request-0
2                      offer-0
3                aid_related-0
4               medical_help-0
5           medical_products-0
6          search_and_rescue-0
7                   security-0
8                   military-0
9                child_alone-0
10                     water-0
11                      food-0
12                   shelter-0
13                  clothing-0
14                     money-0
15            missing_people-0
16                  refugees-0
17                     death-0
18                 other_aid-0
19    infrastructure_related-0
20                 transport-0
21                 buildings-0
22               electricity-0
23                     tools-0
24                 hospitals-0
25                     shops-0
26               aid_centers-0
27      other_infrastructure-0
28           weather_related-0
29                    floods-0
30                     storm-0
31                      fire-0
32      

In [22]:
categories.head(2)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [13]:
c = pd.Index(map(lambda x : str(x)[:-2], df.columns))

In [28]:
c

Index(['', 'messa', 'origin', 'gen', 'categori'], dtype='object')

In [14]:
# select the first row of the categories dataframe
row = categories.iloc[0]
row

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
# category_colnames = pd.Index(map(lambda x : str(x)[:-2], categories.columns))
# print(category_colnames)

0                    related-1
1                    request-0
2                      offer-0
3                aid_related-0
4               medical_help-0
5           medical_products-0
6          search_and_rescue-0
7                   security-0
8                   military-0
9                child_alone-0
10                     water-0
11                      food-0
12                   shelter-0
13                  clothing-0
14                     money-0
15            missing_people-0
16                  refugees-0
17                     death-0
18                 other_aid-0
19    infrastructure_related-0
20                 transport-0
21                 buildings-0
22               electricity-0
23                     tools-0
24                 hospitals-0
25                     shops-0
26               aid_centers-0
27      other_infrastructure-0
28           weather_related-0
29                    floods-0
30                     storm-0
31                      fire-0
32      

In [23]:
getcolname = pd.Index(map(lambda x : str(x)[:-2], row))

In [24]:
getcolname

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [136]:
# # select the first row of the categories dataframe
# row = categories.iloc[0]
# # row
# # type(row)

# type(row[0])
# row[0].split('-')[0]
# len(row)

# # use this row to extract a list of new column names for categories.
# # # one way is to apply a lambda function that takes everything 
# # # # up to the second to last character of each string with slicing
# for i in range(len(row)):
#     category_colnames = row[i].split('-')[0]
# #     categories.loc[:,category_colnames] = ''
#     print(category_colnames)

# print(category_colnames)
# # category_colnames

In [25]:
# rename the columns of `categories`
categories.columns = getcolname
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [26]:
df.head(2)

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...


In [298]:
df.shape

(26248, 5)

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [27]:
row1 = categories.iloc[0]
row1[0]

'related-1'

In [232]:
categories.iloc[2]

related                                  related-1
request                                  request-0
offer                                      offer-0
aid_related                          aid_related-0
medical_help                        medical_help-0
medical_products                medical_products-0
search_and_rescue              search_and_rescue-0
security                                security-0
military                                military-0
child_alone                          child_alone-0
water                                      water-0
food                                        food-0
shelter                                  shelter-0
clothing                                clothing-0
money                                      money-0
missing_people                    missing_people-0
refugees                                refugees-0
death                                      death-0
other_aid                              other_aid-0
infrastructure_related    infra

In [29]:
getvalues = pd.Index(map(lambda x : str(x)[-1:], row1))

In [30]:
getvalues

Index(['1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0'],
      dtype='object')

In [31]:
categories.index

RangeIndex(start=0, stop=26248, step=1)

In [243]:

# for row in categories:
# #     x = categories.iloc[0]
# #     print(i)
#     print(row)
# #     print(categories.iloc[column])
#     # set each value to be the last character of the string
# #     categories[column] = pd.Index(map(lambda x : str(x)[-1:], categories.iloc[x]))
    
# #     # convert column from string to numeric
# #     categories[column] = 
# # categories.head()

related
request
offer
aid_related
medical_help
medical_products
search_and_rescue
security
military
child_alone
water
food
shelter
clothing
money
missing_people
refugees
death
other_aid
infrastructure_related
transport
buildings
electricity
tools
hospitals
shops
aid_centers
other_infrastructure
weather_related
floods
storm
fire
earthquake
cold
other_weather
direct_report


In [32]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    
    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
type(categories['related'][0])

numpy.int64

In [34]:
type(messages['id'][0])

str

In [35]:
#convert id to numeric
# convert column from string to numeric
messages['id'] = pd.to_numeric(messages['id'])

In [36]:
type(messages['id'][0])

numpy.int64

In [37]:
df.head(2)

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...


In [ ]:
# for column in categories:
#     # for column in categories:
#     # set each value to be the last character of the string
#     categories[column] = 
    
#     # convert column from string to numeric
#     categories[column] = 
# categories.head()
#     categories[column] = 
    
#     # convert column from string to numeric
#     categories[column] = 
# categories.head()

In [38]:
categories.tail(10)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
26238,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26239,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
26240,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26242,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26245,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26246,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
26247,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
categories.shape

(26248, 36)

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [40]:
df.head(2)

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...


In [61]:
# drop the original categories column from `df`
df.drop(['categories'], axis=1, inplace=True )

In [42]:
# drop the original categories column from `df`
df.drop(['original'], axis=1, inplace=True )


In [45]:
df.head(2)

,id,message,genre
0,2,Weather update - a cold front from Cuba that c...,direct
1,7,Is the Hurricane over or is it not over,direct


In [46]:
type(messages['id'][0])

numpy.int64

In [47]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df,categories],axis=1)
df.head(2)

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [48]:
#check type of the numbers
type(df['id'])
# type(categories['related'][0])

pandas.core.series.Series

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [49]:
df.head(2)

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [50]:
# check number of duplicates
df.duplicated().any()

True

In [51]:
duplicate = df[df.duplicated()] 

In [52]:
duplicate.count()

id                        32
message                   32
genre                     32
related                   32
request                   32
offer                     32
aid_related               32
medical_help              32
medical_products          32
search_and_rescue         32
security                  32
military                  32
child_alone               32
water                     32
food                      32
shelter                   32
clothing                  32
money                     32
missing_people            32
refugees                  32
death                     32
other_aid                 32
infrastructure_related    32
transport                 32
buildings                 32
electricity               32
tools                     32
hospitals                 32
shops                     32
aid_centers               32
other_infrastructure      32
weather_related           32
floods                    32
storm                     32
fire          

In [53]:
df.shape


(26248, 39)

In [111]:
# drop duplicates
# drop duplicates
df.drop_duplicates().duplicated().any()

False

In [63]:
# check number of duplicates
df.duplicated().any()

True

In [112]:
df.shape

(26248, 40)

In [54]:
#remove duplicate
df =  df.drop_duplicates()

In [55]:
df.shape

(26216, 39)

In [56]:
# check number of duplicates
df.duplicated().any()

False

In [91]:
df.isnull().values.any()

False

In [58]:
df.isnull().sum()

id                        0
message                   0
genre                     0
related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake          

In [93]:
df.drop(['original'], axis=1, inplace=True )


In [122]:
df.isnull().sum()

id                        0
message                   0
genre                     0
related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake          

In [60]:
df.head()

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [98]:
engine = create_engine('sqlite:///disaster-responses.db')
df.to_sql('disaster-responses', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.